###### 2020-10-16 금요일

# Pandas : groupby & pivot

#### 목차

#### 1. `groupby()`
#### 2. `groupby().apply()` / `groupby().agg()`
#### 3. `pd.qcut()` / `pd.cut()`
#### 4. `pivot()` / `pivot_table()`
#### 5. 타이타닉으로 pivot을 실습해보자

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Groupby
   - R의 dplyr패키지의 group_by와 같은 형식으로 작동하는 함수라고 생각하면 편하다.

In [4]:
df = pd.DataFrame({ "학과" : ["컴퓨터","체육교육과","컴퓨터","체육교육과","컴퓨터"],
                    "학년" : [1, 2, 3, 2, 3],
                    "이름" : ["홍길동","김연아","최길동","아이유","신사임당"],
                    "학점" : [1.5, 4.4, 3.7, 4.5, 3.8]})
df

,학과,학년,이름,학점
0,컴퓨터,1,홍길동,1.5
1,체육교육과,2,김연아,4.4
2,컴퓨터,3,최길동,3.7
3,체육교육과,2,아이유,4.5
4,컴퓨터,3,신사임당,3.8


##### 학과를 기준으로 grouping을 한다면?

In [127]:
df['학과'].groupby(df['학과']).groups

{'체육교육과': Int64Index([1, 3], dtype='int64'),
 '컴퓨터': Int64Index([0, 2, 4], dtype='int64')}

##### df.groupby()를 적용하면..
   - 반환되는 타입은 Dataframe이다

In [149]:
df.groupby(df['학과'])

In [150]:
dept_series = df.groupby(df['학과'])
dept_series.get_group('컴퓨터')

,학과,학년,이름,학점
0,컴퓨터,1,홍길동,1.5
2,컴퓨터,3,최길동,3.7
4,컴퓨터,3,신사임당,3.8


In [151]:
dept_series.size()

학과
체육교육과    2
컴퓨터      3
dtype: int64

In [152]:
dept = df.groupby(df['학과']).mean()
dept

,학년,학점
학과,,
체육교육과,2.000000,4.45
컴퓨터,2.333333,3.00


In [24]:
df.groupby(['학과', '학년']).mean()

학점
학과    학년      
체육교육과 2   4.45
컴퓨터   1   1.50
      3   3.75

## 2. apply() / agg()
   - iris 데이터를 이용하여 다음 함수들을 적용해 봅시다
       - groupby().apply()
       - groupby().agg()

In [105]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [29]:
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [32]:
# 각 종별로 가장 큰 값과 가장 작은 값의 비율을 구한다면?
# 파이썬에 비율을 구하는 함수가 존재하지 않기 떄문에
# 아래와 같은 함수를 만들어 준다.

def get_ratio(x):
    return x.max() / x.min()

In [36]:
# 내가 구현한 함수를 groupby에 적용하고자 할때 사용하는 함수
# agg() / apply()

display(iris.groupby(iris['species']).agg(get_ratio))
display(iris.groupby(iris['species']).apply(get_ratio))

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


##### groupby().describe()를 적용하여 각 칼럼의 요약을 볼 수도 있다.

In [41]:
iris.groupby(iris['species']).describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

##### `iris`데이터에서 가장 큰 3개의 `petal_length`를 가지는 행을 출력해보라

In [106]:
def max3_function(df):
    return df.sort_values(by='petal_length', ascending=False)[:3]

In [107]:
iris.groupby(iris['species']).apply(max3_function)


sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

 ##### groupby에서  agg / apply 차이는?
    - agg는 여러 함수를 한번에 적용할 수 있지만
    - apply는 여러 함수를 한번에 적용할 수 없다
    - 이해가 안되면 아래의 코드를 봅시다
        (사실 내부 알고리즘적으로도 다르긴한데.. groupby할때는 agg만 사용한다면 큰 문제 없다)


붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width)등의 평균과 합을 구하라

In [155]:
iris.groupby(iris.species).agg([np.mean, np.sum]).loc[:, ['sepal_length', 'sepal_width']]

sepal_length        sepal_width       
                   mean    sum        mean    sum
species                                          
setosa            5.006  250.3       3.428  171.4
versicolor        5.936  296.8       2.770  138.5
virginica         6.588  329.4       2.974  148.7

In [156]:
iris.groupby(iris.species).apply([np.mean, np.sum]).loc[:, ['sepal_length', 'sepal_width']]

## 오류나쥬?

TypeError: unhashable type: 'list'

## 3. pd.qcut() / pd.cut() 함수

 - transform : 데이터프레임 자체를 변화시키는 함수
 - 원본 프레임과 크기가 같다.
 - 각 붓꽃 꽃잎 길이가 새당 종 내에서 대/중/소 어느것에 해당되는지에 대한 프레임을 만들고 싶다면?
  - cut()
    > 수치형의 칼럼을 동일한 길이의 구간을 기준으로하여 범주화 하는 함수이다.
  - qcut()
    > 수치형의 칼럼을 동일한 개수를 기준으로하여 범주화 하는 함수이다
    
    
###### 이해가 안되면 직접 실습해 보자

In [135]:
def cat3_petal_length(s):
    return pd.qcut(s, 3, labels=['소', '중', '대']).astype(str)

In [157]:
iris['category'] = iris.groupby(iris['species']).petal_length.transform(cat3_petal_length)
iris.head(10)

,sepal_length,sepal_width,petal_length,petal_width,species,category
0,5.1,3.5,1.4,0.2,setosa,소
1,4.9,3.0,1.4,0.2,setosa,소
2,4.7,3.2,1.3,0.2,setosa,소
3,4.6,3.1,1.5,0.2,setosa,중
4,5.0,3.6,1.4,0.2,setosa,소
5,5.4,3.9,1.7,0.4,setosa,대
6,4.6,3.4,1.4,0.3,setosa,소
7,5.0,3.4,1.5,0.2,setosa,중
8,4.4,2.9,1.4,0.2,setosa,소
9,4.9,3.1,1.5,0.1,setosa,중


## 4. pivot() / pivot_table()
   - 데이터프레임에서 두 개의 열을 행과 열의 인덱스 이름으로 사용하여 reshape된 테이블을 의미한다.
   - 새로운 테이블에서 새로운 기준으로 집계
   - pivot(`인덱스로적용할칼럼`, `칼럼으로적용할칼럼`, `값들`)
   - pivot_table(`함수적용할칼럼`, `인덱스로적용할칼럼` `칼럼으로적용할칼럼`, aggfunc=`함수이름`)


In [158]:
import seaborn as sns

In [159]:
titanic = sns.load_dataset('titanic')
titanic.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


In [160]:
titaic_df01 = pd.DataFrame(titanic, columns=['sex', 'pclass'])
titaic_df01

,sex,pclass
0,male,3
1,female,1
2,female,3
3,female,1
4,male,3
...,...,...
886,male,2
887,female,1
888,female,3
889,male,1


##### [데이터프레임].pivot()
   - 성별과 객실 등급에 따른 승객 수 집계한다면?

In [60]:
titaic_df01 = titanic.groupby(['sex', 'pclass']).size().reset_index(name='cnt')
titaic_df01.pivot('sex', 'pclass', 'cnt')

pclass,1,2,3
sex,,,
female,94,76,144
male,122,108,347


##### [데이터프레임].pivot()
   - 성별과 생존여부에 따른 승객 수를 집계한다면?

In [183]:
df = titanic.groupby(['sex', 'survived']).size().reset_index(name='cnt')
df.pivot('sex', 'survived', 'cnt')

survived,0,1
sex,,
female,81,233
male,468,109


##### [데이터프레임].pivot_table()
   - 성별과 생존여부에 따른 승객 수를 집계한다면?

In [184]:
titanic['cnt'] = 1
titanic.pivot_table('cnt', 'sex', 'survived', aggfunc=np.sum)

survived,0,1
sex,,
female,81,233
male,468,109


##### `groupby`와 `pivot`함수를 실습해 봅시다

In [186]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
pivot_sample_df = pd.DataFrame(data, columns=columns)
pivot_sample_df


,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


##### pivot_table() 함수의 옵션
 - aggfunc : 집계 함수
 - fill_value : NaN 대체값
 - margins : 행과 열의 합을 구해줄 것이냐? (True / False)
 - margins_name : 마진 열(행)의 이름

In [ ]:
pivot_sample_df.pivot_table('인구', '도시', '연도',
                           margins=)

In [187]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [188]:
# 식사대금 대비 탑의 비율이 어떤 경우에 가장 높은지를 찾는다면?
tips['tip/pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip/pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [189]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size,tip/pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [199]:
tips.groupby('sex').size()

sex
Male      157
Female     87
dtype: int64

In [215]:
# 성별과 흡연유무로 나누어서 데이터의 갯수를 세어보자면??
display(tips.groupby(['sex', 'smoker']).count())
display(tips.groupby(['sex', 'smoker']).mean())

total_bill  tip  day  time  size  tip/pct
sex    smoker                                           
Male   Yes             60   60   60    60    60       60
       No              97   97   97    97    97       97
Female Yes             33   33   33    33    33       33
       No              54   54   54    54    54       54

total_bill       tip      size   tip/pct
sex    smoker                                          
Male   Yes      22.284500  3.051167  2.500000  0.152771
       No       19.791237  3.113402  2.711340  0.160669
Female Yes      17.977879  2.931515  2.242424  0.182150
       No       18.105185  2.773519  2.592593  0.156921

In [216]:
display(tips.head())
# aggfunc='count'이기 때문에 첫번째 함수요소에 어떠한 칼럼이 와도 상관 없다.
# 만약 aggfunc='mean'과 같은 함수라면 함수의 첫번째 요소에 들어갈 칼럼이 중요할 것이다.
# margins=True는 행과 열의 총합을 구해주냐 아니냐의 여부이다
display(tips.pivot_table('day', 'sex', 'smoker', aggfunc='count', margins=True))
display(tips.pivot_table('time', 'sex', 'smoker', aggfunc='count', margins=True))
display(tips.pivot_table('total_bill', 'sex', 'smoker', aggfunc='count', margins=False))


,total_bill,tip,sex,smoker,day,time,size,tip/pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


smoker,Yes,No
sex,,
Male,60,97
Female,33,54


In [233]:
# 성별과 흡연 여부에 따른 평균 팁의 비율을 살펴본다면?

# 1. pivot_table
display(tips.pivot_table('tip/pct', 'sex', 'smoker', aggfunc='mean', margins=False))

# 2 pivot
df = tips.groupby(['sex' , 'smoker']).mean().loc[:, ['tip/pct']]
display(df)
df2 = df.reset_index()
df2.pivot('sex', 'smoker', 'tip/pct')

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


tip/pct
sex    smoker          
Male   Yes     0.152771
       No      0.160669
Female Yes     0.182150
       No      0.156921

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


In [236]:
# 원하는 칼럼으로 groupby한 후 통계량을 확인하려면??
display(tips.groupby(['sex', 'smoker']).describe())
display(tips.groupby(['sex', 'smoker']).describe().T) # 이게더 보기 편하쥬?

total_bill                                                      \
                   count       mean       std   min      25%    50%      75%   
sex    smoker                                                                  
Male   Yes          60.0  22.284500  9.911845  7.25  15.2725  20.39  28.5725   
       No           97.0  19.791237  8.726566  7.51  13.8100  18.24  22.8200   
Female Yes          33.0  17.977879  9.189751  3.07  12.7600  16.27  22.1200   
       No           54.0  18.105185  7.286455  7.25  12.6500  16.69  20.8625   

                       tip            ... size      tip/pct            \
                 max count      mean  ...  75%  max   count      mean   
sex    smoker                         ...                               
Male   Yes     50.81  60.0  3.051167  ...  3.0  5.0    60.0  0.152771   
       No      48.33  97.0  3.113402  ...  3.0  6.0    97.0  0.160669   
Female Yes     44.30  33.0  2.931515  ...  2.0  4.0    33.0  0.182150   
       No      35.83  54.0  2.773519  ...  3.0  6.0    54.0  0.156921   

                                                                           
                    std       min       25%       50%       75%       max  
sex    smoker                                                              
Male   Yes     0.090588  0.035638  0.101845  0.141015  0.191697  0.710345  
       No      0.041849  0.071804  0.131810  0.157604  0.186220  0.291990  
Female Yes     0.071595  0.056433  0.152439  0.173913  0.198216  0.416667  
       No      0.036421  0.056797  0.139708  0.149691  0.181630  0.252672  

[4 rows x 32 columns]

sex                    Male                Female           
smoker                  Yes         No        Yes         No
total_bill count  60.000000  97.000000  33.000000  54.000000
           mean   22.284500  19.791237  17.977879  18.105185
           std     9.911845   8.726566   9.189751   7.286455
           min     7.250000   7.510000   3.070000   7.250000
           25%    15.272500  13.810000  12.760000  12.650000
           50%    20.390000  18.240000  16.270000  16.690000
           75%    28.572500  22.820000  22.120000  20.862500
           max    50.810000  48.330000  44.300000  35.830000
tip        count  60.000000  97.000000  33.000000  54.000000
           mean    3.051167   3.113402   2.931515   2.773519
           std     1.500120   1.489559   1.219916   1.128425
           min     1.000000   1.250000   1.000000   1.000000
           25%     2.000000   2.000000   2.000000   2.000000
           50%     3.000000   2.740000   2.880000   2.680000
           75%     3.820000   3.710000   3.500000   3.437500
           max    10.000000   9.000000   6.500000   5.200000
size       count  60.000000  97.000000  33.000000  54.000000
           mean    2.500000   2.711340   2.242424   2.592593
           std     0.892530   0.989094   0.613917   1.073146
           min     1.000000   2.000000   1.000000   1.000000
           25%     2.000000   2.000000   2.000000   2.000000
           50%     2.000000   2.000000   2.000000   2.000000
           75%     3.000000   3.000000   2.000000   3.000000
           max     5.000000   6.000000   4.000000   6.000000
tip/pct    count  60.000000  97.000000  33.000000  54.000000
           mean    0.152771   0.160669   0.182150   0.156921
           std     0.090588   0.041849   0.071595   0.036421
           min     0.035638   0.071804   0.056433   0.056797
           25%     0.101845   0.131810   0.152439   0.139708
           50%     0.141015   0.157604   0.173913   0.149691
           75%     0.191697   0.186220   0.198216   0.181630
           max     0.710345   0.291990   0.416667   0.252672

## 5. 타이타닉으로 pivot을 실습해보자
1. qcut 명령으로 세 개의 나이 그룹을 만든다.
 - A, B, C
 
2. 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
 - 행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 
 - 선실 인덱스를 사용한다. 
 - 생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다.

3. 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.



In [4]:
import seaborn as sns

titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


##### 1. qcut 명령으로 세 개의 나이 그룹을 만든다.
   - A, B, C

In [99]:
titanic['age_class'] = pd.qcut(titanic['age'], 3, labels=['A', 'B', 'C'])
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_class
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,A
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,C
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,B
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,C
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,C


##### 2. 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
 - 행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다. 
 - 생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다.


In [108]:
count_df = titanic.pivot_table('survived', ['sex', 'age_class'], 'pclass', aggfunc='sum', margins=True)

all_count = titanic.shape[0]
survival_rate_df = count_df / all_count

display(count_df)
display(survival_rate_df)


pclass              1   2   3  All
sex    age_class                  
female A           21  20  30   71
       B           18  30  13   61
       C           43  18   4   65
male   A            5  10  17   32
       B           11   3  18   32
       C           24   2   3   29
All               122  83  85  290

pclass                   1         2         3       All
sex    age_class                                        
female A          0.023569  0.022447  0.033670  0.079686
       B          0.020202  0.033670  0.014590  0.068462
       C          0.048260  0.020202  0.004489  0.072952
male   A          0.005612  0.011223  0.019080  0.035915
       B          0.012346  0.003367  0.020202  0.035915
       C          0.026936  0.002245  0.003367  0.032548
All               0.136925  0.093154  0.095398  0.325477

##### 3. 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.

In [109]:
count_df = titanic.pivot_table('survived', 'sex', 'pclass', aggfunc='sum', margins=True)
all_count = titanic.shape[0]

suvival_rate_df = count_df / all_count

display(suvival_rate_df)

pclass,1,2,3,All
sex,,,,
female,0.102132,0.078563,0.080808,0.261504
male,0.050505,0.019080,0.052750,0.122334
All,0.152637,0.097643,0.133558,0.383838
